In [10]:
import time
import os
import logging

import streamlit as st
import faiss
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable.passthrough import RunnablePassthrough
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

# Add more agents
from langchain.agents import create_tool_calling_agent, initialize_agent
from langchain.agents import AgentExecutor
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

In [2]:
def load_and_split_pdf(file_path, chunk_size=1000, chunk_overlap=100):
    """
    Load a PDF file and split it into chunks.
    """
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

def create_vector_store_qdrant(doc_chunks, embeddings):
    """
    Create and populate a Qdrant vector store with document chunks.
    """
    # Initialize Qdrant client
    qdrant_client = QdrantClient(host="10.100.140.54", port=6333)  # Adjust host and port as needed

    # Define the collection name
    collection_name = "document_chunks"

    # Create a collection in Qdrant
    qdrant_client.recreate_collection(
        collection_name=collection_name,
        vectors_config=VectorParams(size=len(embeddings.embed_query("sample text")), distance=Distance.COSINE)
    )

    # Prepare vectors and payloads
    vectors = []
    payloads = []
    for i, chunk in enumerate(doc_chunks):
        vector = embeddings.embed_query(chunk.page_content)
        vectors.append(vector)
        payloads.append({"id": i, "text": chunk.page_content})

    # Upload vectors to Qdrant
    qdrant_client.upload_collection(
        collection_name=collection_name,
        vectors=vectors,
        payload=payloads
    )

    return qdrant_client, collection_name

In [3]:
def build_chatbot_prompt_template():
    prompt_text = """
        You are a helpful assistant answering questions based on the provided context from uploaded documents. Your name is Detnam.

        Conversation history (latest messages appear last):
        {history}

        New question:
        {question}

        Retrieved context:
        {context}
        
        Thought process:
        {agent_scratchpad}

        Your response:
    """
    return ChatPromptTemplate.from_template(prompt_text)

def format_documents(documents):
    """
    Format a list of documents into a single string with separated contents.
    """
    return "\n\n".join([doc.page_content for doc in documents])

In [4]:
def process_question_with_history_qdrant(uploaded_files, question, history, embeddings, chat_model):
    combined_chunks = []

    # Process uploaded files and extract chunks
    for uploaded_file in uploaded_files:
        temp_file_path = f"temp_{uploaded_file.name}"
        with open(temp_file_path, "wb") as f:
            f.write(uploaded_file.read())

        doc_chunks = load_and_split_pdf(temp_file_path)
        combined_chunks.extend(doc_chunks)
        os.remove(temp_file_path)

    # Create a vector store
    qdrant_client, collection_name = create_vector_store_qdrant(combined_chunks, embeddings)

    # Retrieve relevant documents
    query_vector = embeddings.embed_query(question)
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=3  # Adjust the number of results as needed
    )

    # Format the retrieved documents
    retrieved_docs = [hit.payload["text"] for hit in search_result]
    formatted_context = "\n\n".join(retrieved_docs)

    # Prepare the pipeline
    prompt_template = build_chatbot_prompt_template()

    # Ensure each part of the pipeline is correctly set up
    rag_pipeline = (
        RunnablePassthrough()
        | {
            "context": lambda x: formatted_context,  # Use a lambda to pass the context
            "question": lambda x: question,          # Use a lambda to pass the question
            "history": lambda x: history,            # Use a lambda to pass the history
            "agent_scratchpad": lambda x: ""
        }
        | prompt_template
        | chat_model
        | StrOutputParser()
    )
    
    input_dict = {
        "context": formatted_context,
        "question": question,
        "history": history,
        "agent_scratchpad": "",
    }
    logging.debug(f"Input to rag_pipeline.invoke: {input_dict}")
    result = rag_pipeline.invoke(input_dict)
    logging.debug(f"Result from rag_pipeline.invoke: {result}")

    return result

In [5]:
# Example usage create agent
@tool
def search(query: str) -> str:
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    
    search = TavilySearchResults(
        max_results=5,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=True,
    )
    response = search.invoke(query)
    return response

In [6]:
embedding_model_name = st.sidebar.text_input("Embedding Model", value="nomic-embed-text")
model_url = st.sidebar.text_input("Model Base URL", value="http://localhost:11434")
embeddings = OllamaEmbeddings(model=embedding_model_name, base_url=model_url)
chat_model = ChatOllama(model="llama3", base_url=model_url)

tools =  [search]
prompt_template = build_chatbot_prompt_template()
agent = create_tool_calling_agent(llm=chat_model, tools=tools, prompt=prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

2025-01-20 21:33:55.596 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 21:33:55.597 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 21:33:55.598 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 21:33:55.599 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-01-20 21:33:55.599 WARNING streamlit.runtime.state.session_state_proxy: Session state does not function when running a script without `streamlit run`
2025-01-20 21:33:55.600 WARNING streamlit.runtime.scriptrunner_utils.script_run_c

In [7]:
# Load file dataset
file_pdf = "load_file.pdf"
docs = load_and_split_pdf(file_pdf)

# Test query vector database
question = "How to deploy backend"
qdrant_client, collection_name = create_vector_store_qdrant(docs, embeddings)
query_vector = embeddings.embed_query(question)
search_result = qdrant_client.search(collection_name=collection_name, query_vector=query_vector, limit=3)
print(search_result)

/tmp/ipykernel_694715/4058644294.py:21: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant_client.recreate_collection(


[ScoredPoint(id='37b96fda-75b1-48c8-a7ee-609310778632', version=0, score=0.5997566, payload={'id': 0, 'text': 'User đăng nhập server\nTài khoản deploy\nTên đăng nhập: deployuser\nMật khẩu: tmt@dxt2024\nCác thư mục\nCode trên máy tính cá nhân\nD:\\coding\\dxtech\nThư mục chứa SSL\n/home/staging/deploy/dxtech_ssl\nThư mục chứa code backend\n/home/staging/deploy/dxtech-service/.. : các service code theo kiến trúc micro service\n/home/staging/deploy/faceid-backend: customer monitoring chưa có micro service\n/home/staging/deploy/ekyc-backend: ekyc chưa có micro service\nĐường dẫn chứa các file service\nsudo vi /etc/systemd/system/…\nThư mục chứa code frontend\n/home/staging/deploy/dxtech-frontend/..: các code frontend ởrepo mới\n/home/staging/deploy//..: các code frontend ởrepo mới\nThư mục chứa file config deploy frontend với nginx\n/etc/nginx/sites-enabled\nekyc: web ekyc\nface-recognition: customer-monitoring\ntest-faceid: customer-monitoring bản test'}, vector=None, shard_key=None, orde

/tmp/ipykernel_694715/2212122181.py:9: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_result = qdrant_client.search(collection_name=collection_name, query_vector=query_vector, limit=3)


In [8]:
# Format the retrieved documents
retrieved_docs = [hit.payload["text"] for hit in search_result]
formatted_context = "\n\n".join(retrieved_docs)

formatted_context

'User đăng nhập server\nTài khoản deploy\nTên đăng nhập: deployuser\nMật khẩu: tmt@dxt2024\nCác thư mục\nCode trên máy tính cá nhân\nD:\\coding\\dxtech\nThư mục chứa SSL\n/home/staging/deploy/dxtech_ssl\nThư mục chứa code backend\n/home/staging/deploy/dxtech-service/.. : các service code theo kiến trúc micro service\n/home/staging/deploy/faceid-backend: customer monitoring chưa có micro service\n/home/staging/deploy/ekyc-backend: ekyc chưa có micro service\nĐường dẫn chứa các file service\nsudo vi /etc/systemd/system/…\nThư mục chứa code frontend\n/home/staging/deploy/dxtech-frontend/..: các code frontend ởrepo mới\n/home/staging/deploy//..: các code frontend ởrepo mới\nThư mục chứa file config deploy frontend với nginx\n/etc/nginx/sites-enabled\nekyc: web ekyc\nface-recognition: customer-monitoring\ntest-faceid: customer-monitoring bản test\n\nGhi log\n/opt/dxt-ekyc-logs/..\nEureka server\n/home/staging/deploy/dxtech-service/eureka-server\nAPI gateway\n/home/staging/deploy/dxtech-serv

In [9]:
prompt_template = build_chatbot_prompt_template()
prompt_template

ChatPromptTemplate(input_variables=['agent_scratchpad', 'context', 'history', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'context', 'history', 'question'], input_types={}, partial_variables={}, template='\n        You are a helpful assistant answering questions based on the provided context from uploaded documents. Your name is Detnam.\n\n        Conversation history (latest messages appear last):\n        {history}\n\n        New question:\n        {question}\n\n        Retrieved context:\n        {context}\n        \n        Thought process:\n        {agent_scratchpad}\n\n        Your response:\n    '), additional_kwargs={})])

In [18]:
tools =  [search]
agent = create_tool_calling_agent(llm=chat_model, tools=tools, prompt=prompt_template)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

ImportError: cannot import name 'create_agent' from 'langchain.agents' (/home/namnh1/miniconda3/envs/rag/lib/python3.10/site-packages/langchain/agents/__init__.py)

In [13]:
history = ""

In [14]:
rag_pipeline = (
    RunnablePassthrough()
    | {
        "agent_scratchpad": lambda x: "",
        "context": lambda x: formatted_context,  # Use a lambda to pass the context
        "history": lambda x: history,            # Use a lambda to pass the history
        "question": lambda x: question,          # Use a lambda to pass the question
    }
    | prompt_template
    | agent_executor
    | StrOutputParser()
)

In [15]:
input_dict = {
    "agent_scratchpad": "",
    "context": formatted_context,
    "history": history,
    "question": question
}

In [16]:
input_dict

{'agent_scratchpad': '',
 'context': 'User đăng nhập server\nTài khoản deploy\nTên đăng nhập: deployuser\nMật khẩu: tmt@dxt2024\nCác thư mục\nCode trên máy tính cá nhân\nD:\\coding\\dxtech\nThư mục chứa SSL\n/home/staging/deploy/dxtech_ssl\nThư mục chứa code backend\n/home/staging/deploy/dxtech-service/.. : các service code theo kiến trúc micro service\n/home/staging/deploy/faceid-backend: customer monitoring chưa có micro service\n/home/staging/deploy/ekyc-backend: ekyc chưa có micro service\nĐường dẫn chứa các file service\nsudo vi /etc/systemd/system/…\nThư mục chứa code frontend\n/home/staging/deploy/dxtech-frontend/..: các code frontend ởrepo mới\n/home/staging/deploy//..: các code frontend ởrepo mới\nThư mục chứa file config deploy frontend với nginx\n/etc/nginx/sites-enabled\nekyc: web ekyc\nface-recognition: customer-monitoring\ntest-faceid: customer-monitoring bản test\n\nGhi log\n/opt/dxt-ekyc-logs/..\nEureka server\n/home/staging/deploy/dxtech-service/eureka-server\nAPI gate

In [ ]:
result = rag_pipeline.invoke(input_dict)
result

In [12]:
from langchain_community.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

search.invoke("What is today's stock market news?")

/home/namnh1/miniconda3/envs/rag/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


"Nasdaq futures plunged on Monday to lead a stock rout on Wall Street as a Chinese startup rattled faith in US leadership and profitability in AI, taking a hammer to Nvidia and other Big Tech stocks. Find the latest stock market news from every corner of the globe at Reuters.com, your online source for breaking international market and finance news AP AUDIO: Stock market today: Wall Street rallies toward its best week since Trump's election The AP's Seth Sutel reports stocks keep moving higher. The Magnificent Seven have been under pressure recently because of criticism their stock prices may have shot too high after leading the market for so many years. Stock Market News And Analysis. The analysis you'll find in the Stock Market Today is based on over 130 years of market history and a detailed study of every top-performing stock since the 1880s. Stocks surged on Wednesday after the latest consumer price index report showed core inflation unexpectedly slowed in December."

In [13]:
import time
import os
import logging

import streamlit as st
import faiss
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.schema.runnable.passthrough import RunnablePassthrough
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

# Add more agents
from langchain.agents import create_tool_calling_agent, initialize_agent
from langchain.agents import AgentExecutor
from langchain_community.tools import TavilySearchResults
from langchain_core.tools import tool

In [14]:
from langchain_ollama import ChatOllama 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough 
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.tools import tool


llm = ChatOllama(model='llama3.1', base_url='http://localhost:11434')
llm.invoke('hi')

AIMessage(content="How's it going? Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-01-27T14:42:03.69912351Z', 'done': True, 'done_reason': 'stop', 'total_duration': 339486338, 'load_duration': 39279293, 'prompt_eval_count': 11, 'prompt_eval_duration': 26000000, 'eval_count': 21, 'eval_duration': 272000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-610b4e42-fc18-46c6-be6f-73ecfc96628f-0', usage_metadata={'input_tokens': 11, 'output_tokens': 21, 'total_tokens': 32})

In [15]:
@tool
def search(query: str) -> str:
    """
    Search the web for realtime and latest information.
    for examples, news, stock market, weather updates etc.
    
    Args:
    query: The search query
    """
    
    search = DuckDuckGoSearchRun(
    )
    response = search.invoke(query)
    return response

In [16]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS 
from langchain_community.docstore.in_memory import InMemoryDocstore

In [17]:
def load_and_split_pdf(file_path, chunk_size=1000, chunk_overlap=100):
    """
    Load a PDF file and split it into chunks.
    """
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    return splitter.split_documents(documents)

In [18]:
embeddings = OllamaEmbeddings(model='nomic-embed-text', base_url='http://localhost:11434')

file_path = "load_file.pdf"

doc_chunks = load_and_split_pdf(file_path)

sample_vector = embeddings.embed_query("sample text")
index = faiss.IndexFlatL2(len(sample_vector))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)
vector_store.add_documents(doc_chunks)

['e28f023e-2eb4-445f-871f-3a3627c6a553',
 '5ef07e30-caa2-4f5d-bb01-314d31363da4',
 '0b0dc12b-9324-4f13-b6ee-b77b0b2369f4',
 '742b3bc2-c034-4e65-b6f0-9125c854c42c']

In [19]:
retriever = vector_store.as_retriever(search_type = 'similarity', 
                                      search_kwargs = {'k': 3})

In [20]:
tools = [search]
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
agent = create_tool_calling_agent(llm, tools, prompt)

/home/namnh1/miniconda3/envs/rag/lib/python3.10/site-packages/langsmith/client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [21]:
def build_chatbot_prompt_template():
    prompt_text = """
        You are Detnam, a knowledgeable and helpful assistant. You answer questions using information from provided documents and context. Be clear, concise, and engaging in your responses.

        Conversation History (most recent messages at the end):
        {history}

        User Question:
        {question}

        Relevant Context (retrieved from documents):
        {context}
        
        Thought Process (reason through your answer):
        {agent_scratchpad}

        Please provide a well-structured, informative, and friendly response:
    """
    return ChatPromptTemplate.from_template(prompt_text)

prompt = build_chatbot_prompt_template()

In [22]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [23]:
question = "how to deploy backend?"
response = agent_executor.invoke({'input': question})

--------------------------------------------------


> Entering new AgentExecutor chain...

Invoking: `search` with `{'query': 'deploying a backend server'}`




/home/namnh1/miniconda3/envs/rag/lib/python3.10/site-packages/langchain_community/utilities/duckduckgo_search.py:64: UserWarning: backend='api' is deprecated, using backend='auto'
  ddgs_gen = ddgs.text(


How to deploy node express app on vercel free step by step guide. Introduction. Deploying your Node.js backend project can seem daunting, but with platforms like Vercel, the process is streamlined ... To deploy a Node.js backend to Vercel, you can follow the simple approach that involves setting up your Node.js project, configuring necessary files, and using the Vercel CLI. Vercel is known for its ease of use and automatic optimizations, making it an excellent choice for deploying full-stack appl In this tutorial, you will learn how to deploy or host your backend project using Vercel serverless function via GitHub. Before we dive deep into building and configuration, let's understand the concept of serverless function in Vercel. Transitioning from a traditional Express.js server to build-in Vercel Functions. Taking this step means you can stop configuring your own server from scratch and let Vercel manage that for you. Transitioning to Next.js if you are migrating from an Express.js Se

In [24]:
print(response['output'])

It seems like the tool call response was able to provide a detailed step-by-step guide on how to deploy a backend server using Vercel.

To summarize:

1. Set up your Node.js project.
2. Configure necessary files.
3. Use the Vercel CLI.
4. Trigger manual initial deployment.
5. Configure Environment Variables.

Additionally, it covers transitioning from traditional Express.js servers to build-in Vercel Functions and migrating from Express.js Server and React SPA to Next.js.

Please let me know if you have any further questions or concerns!
